In [9]:
import pandas as pd

# Load the dataset
file_path = 'FoodContents.csv'  # Replace with your file path
data = pd.read_csv(file_path)

# Basic statistics
print(data.describe())


         energy_kj  energy_kcal       carb_g    protein_g        fat_g  \
count  1014.000000  1014.000000  1014.000000  1014.000000  1014.000000   
mean    971.821321   233.735897    18.354004     4.753008    16.267584   
std     773.154509   187.599467    16.745936     3.442746    20.452322   
min      27.670000     6.610000     0.000000     0.000000     0.000000   
25%     423.155000   101.180000     5.582500     2.100000     4.030000   
50%     731.235000   174.920000    11.925000     3.990000     9.095000   
75%    1317.600000   315.845000    26.870000     6.602500    17.085000   
max    3453.320000   839.330000    86.530000    21.550000    90.450000   

       freesugar_g      fibre_g        sfa_mg       mufa_mg       pufa_mg  \
count  1014.000000  1014.000000   1014.000000   1014.000000   1014.000000   
mean      8.824832     1.962485   4281.361529   3925.130128   6519.899980   
std      12.378324     2.771622   4408.693905   4929.256232  13330.837512   
min       0.000000     0.

In [5]:
def calculate_daily_calories(weight, height, age, activity_level='moderate', gender='male'):
    """Calculate daily calorie needs using Mifflin-St Jeor Equation."""
    if gender == 'male':
        bmr = 10 * weight + 6.25 * height - 5 * age + 5
    else:
        bmr = 10 * weight + 6.25 * height - 5 * age - 161
    
    # Adjust based on activity level
    activity_multiplier = {
        'sedentary': 1.2,
        'light': 1.375,
        'moderate': 1.55,
        'active': 1.725
    }
    return bmr * activity_multiplier[activity_level]

# Example input
current_weight = 65  # kg
target_weight = 60  # kg
height = 170  # cm
age = 25  # years
daily_calories = calculate_daily_calories(current_weight, height, age)
print(f"Daily calorie needs for maintenance: {daily_calories:.2f} kcal")


Daily calorie needs for maintenance: 2468.38 kcal


In [10]:
import numpy as np

def generate_diet_plan(data, daily_calories, num_days=30):
    """Generate a 30-day diet plan."""
    diet_plan = []
    food_items = data['food_name'].tolist()
    for day in range(num_days):
        # Randomly select food items until calorie goal is met
        day_plan = []
        total_calories = 0
        while total_calories < daily_calories:
            food = np.random.choice(food_items)
            if food not in day_plan:  # Avoid repeats within the day
                nutrients = data[data['food_name'] == food].iloc[0]
                day_plan.append({
                    'Food': food,
                    'Calories': nutrients['energy_kcal'],
                    'Proteins': nutrients['protein_g'],
                    'Carbs': nutrients['carb_g'],
                    'Fats': nutrients['fat_g']
                })
                total_calories += nutrients['energy_kcal']
        diet_plan.append(day_plan)
    return diet_plan

# Generate diet plan
diet_plan = generate_diet_plan(data, daily_calories - 500)  # Assuming a 500 kcal deficit
for i, day in enumerate(diet_plan[:3]):  # Show first 3 days
    print(f"Day {i+1}: {day}")


Day 1: [{'Food': 'Apple jelly (Seb ki jelly)', 'Calories': 152.13, 'Proteins': 0.1, 'Carbs': 39.37, 'Fats': 0.21}, {'Food': 'Coconut kheer (Nariyal ki kheer)', 'Calories': 162.61, 'Proteins': 3.4, 'Carbs': 14.42, 'Fats': 10.55}, {'Food': 'Green chickpeas pulao (Choliya pulao/Hare chane ka pulao)', 'Calories': 173.43, 'Proteins': 4.88, 'Carbs': 27.57, 'Fats': 4.82}, {'Food': 'Apple snowballs', 'Calories': 101.51, 'Proteins': 0.86, 'Carbs': 24.58, 'Fats': 0.31}, {'Food': 'Gum icing', 'Calories': 223.07, 'Proteins': 5.94, 'Carbs': 53.19, 'Fats': 0.0}, {'Food': 'Christmas biscuit', 'Calories': 406.07, 'Proteins': 5.67, 'Carbs': 60.97, 'Fats': 15.88}, {'Food': 'Pineapple jam ', 'Calories': 149.18, 'Proteins': 0.32, 'Carbs': 38.27, 'Fats': 0.15}, {'Food': 'Caramel ice cream', 'Calories': 173.52, 'Proteins': 4.31, 'Carbs': 14.81, 'Fats': 11.15}, {'Food': 'Mango raita (Aam ka raita)', 'Calories': 67.37, 'Proteins': 3.9, 'Carbs': 8.23, 'Fats': 2.14}, {'Food': 'Peas kachori (Matar kachori)', 'Ca

In [16]:
def adjust_diet_plan(data, diet_plan, eaten_food):
    """
    Adjust the diet plan if the user eats something not in the plan for that day.
    """
    updated_diet_plan = []

    for day in diet_plan:
        updated_day = []
        food_replaced = False

        for meal in day:
            if meal['Food'] == eaten_food and not food_replaced:
                # Replace the consumed food with a healthier alternative
                substitute = data.sample(1).iloc[0]
                new_food = {
                    'Food': substitute['food_name'],
                    'Calories': substitute['energy_kcal'],
                    'Proteins': substitute['protein_g'],
                    'Carbs': substitute['carb_g'],
                    'Fats': substitute['fat_g']
                }
                updated_day.append(new_food)
                food_replaced = True
            else:
                updated_day.append(meal)
        
        updated_diet_plan.append(updated_day)
    
    return updated_diet_plan


In [17]:
current_weight = float(input("Enter your current weight (kg): "))
target_weight = float(input("Enter your target weight (kg): "))
height = float(input("Enter your height (cm): "))
age = int(input("Enter your age: "))
gender = input("Enter your gender (male/female): ").lower()
activity_level = input("Enter your activity level (sedentary/light/moderate/active): ").lower()

# Step 2: Calculate calorie requirements
daily_calories = calculate_daily_calories(current_weight, height, age, activity_level, gender)
calorie_deficit = 500  # Target daily deficit
target_calories = daily_calories - calorie_deficit

print(f"Target daily calorie intake: {target_calories:.2f} kcal")

# Step 3: Generate initial diet plan
diet_plan = generate_diet_plan(data, target_calories)
print("\nGenerated Diet Plan for the First 3 Days:")
for i, day in enumerate(diet_plan[:3]):
    print(f"Day {i+1}:")
    for meal in day:
        print(f"  {meal['Food']} - {meal['Calories']} kcal")

# Step 4: Daily feedback loop
while True:
    print("\n--- Daily Feedback ---")
    day_number = int(input("Enter the day you are on (1-30): "))
    print("Your diet plan for today:")
    for meal in diet_plan[day_number - 1]:
        print(f"  {meal['Food']} - {meal['Calories']} kcal")
    
    eaten_food = input("Enter any food you ate today (or 'none' if you followed the plan): ").strip()
    if eaten_food.lower() == 'none':
        print("Great! No changes to your diet plan.")
    else:
        print(f"Adjusting diet plan to account for {eaten_food}...")
        diet_plan = adjust_diet_plan(data, diet_plan, eaten_food)
        print("Updated diet plan:")
        for i, day in enumerate(diet_plan[:3]):
            print(f"Day {i+1}:")
            for meal in day:
                print(f"  {meal['Food']} - {meal['Calories']} kcal")

    continue_feedback = input("Do you want to continue testing? (yes/no): ").lower()
    if continue_feedback != 'yes':
        break


Target daily calorie intake: 2321.00 kcal

Generated Diet Plan for the First 3 Days:
Day 1:
  Garlic chickpea soup (Lahasun aur chane ka shoraba) - 89.53 kcal
  Kidney bean sandwich with cottage cheese - 176.64 kcal
  Chilli chicken - 198.83 kcal
  Soyabean namak paras - 838.46 kcal
  Apple kheer (Seb ki kheer) - 81.3 kcal
  Minced meat patties - 265.72 kcal
  Finger millet biscuit (Ragi biscuit) - 458.56 kcal
  Raspberry shortbread cookies - 441.12 kcal
Day 2:
  Apple mousse - 106.92 kcal
  Cracked wheat porridge (Meetha daliya) - 81.57 kcal
  Cold summer garden soup - 49.19 kcal
  Cabbage kheer (Pattagobhi ki kheer) - 83.51 kcal
  Semolina milk drink (Thari kanji) - 44.16 kcal
  Semolina ladoo with coconut (Suji/Rava aur nariyal ke ladoo ) - 464.43 kcal
  Plain pulao - 140.21 kcal
  Sprouted moong parantha/paratha - 228.59 kcal
  Chocochip ice cream - 241.45 kcal
  Finger millet biscuit (Ragi biscuit) - 458.56 kcal
  Lemon meringue pie - 224.43 kcal
  Chinese cabbage and meat ball so

ValueError: invalid literal for int() with base 10: 'no'